In [4]:
!pip3 install sh

In [5]:
import os
from sh import gunzip

## Data Preperation
To prepare the data for StageNet, we need to download the data an process it using MIMIC-III Benchmark to procduce the decompensation data.

### Download
To obtain the full MIMIC-III dataset, download the files from physionet.org using your credentialed account. First set an environment variable for the username and password.

In [1]:
!wget -r -N -c -np --user <username> --password <password> https://physionet.org/files/mimiciii/1.4/

zsh:1: no such file or directory: username


### Extract Files
Once the file as are obtained, extact all the files.

In [6]:
MIMIC3_DIR = './physionet.org/files/mimiciii/1.4/'
for filename in os.listdir(MIMIC3_DIR):
    f = os.path.join(MIMIC3_DIR, filename)
    if os.path.isfile(f):
        gunzip(f)

FileNotFoundError: [Errno 2] No such file or directory: './physionet.org/files/mimiciii/1.4/'

## Data Processing

### Clone the repo
Copy the original benchmark code to run the steps on the extracted CSV files

In [ ]:
!git clone https://github.com/YerevaNN/mimic3-benchmarks.git
%cd mimic3-benchmarks

### Install dependencies
The code uses python 3.7. The following steps run using python3.7, assuming it is installed properly and the executable is present the user bin directory.

In [ ]:
!pip3.7 install -r requirements.txt

### Step 1: Generate subject directories
This step generates one directory per SUBJECT_ID and writes ICU stay information to data/{SUBJECT_ID}/stays.csv, diagnoses to data/{SUBJECT_ID}/diagnoses.csv, and events to data/{SUBJECT_ID}/events.csv.

In [ ]:
!python3.7 -m mimic3benchmark.scripts.extract_subjects ../physionet.org/files/mimiciii/1.4/ data/root/

### Step 2: Data cleaning
This step attempts to fix some issues with each subject ID data. Some of the fixes:
- Remove all events for which HADM_ID is missing.
- Remove all events for which HADM_ID is not present in stays.csv.
- If ICUSTAY_ID is missing in an event and HADM_ID is not missing, then we look at stays.csv and try to recover ICUSTAY_ID.
- Remove all events for which we cannot recover ICUSTAY_ID.
- Remove all events for which ICUSTAY_ID is not present in stays.csv.

In [ ]:
!python3.7 -m mimic3benchmark.scripts.validate_events data/root/

### Step 3: Generate episodes time series
This step breaks up each subject ICU stays into individual time seriese. Time series of events are stored in {SUBJECT_ID}/episode{#}_timeseries.csv (where # counts distinct episodes) while episode-level information (patient age, gender, ethnicity, height, weight) and outcomes (mortality, length of stay, diagnoses) are stores in {SUBJECT_ID}/episode{#}.csv

In [ ]:
!python3.7 -m mimic3benchmark.scripts.extract_episodes_from_subjects data/root/

### Step 4: Split traning/testing data
This step splits the whole dataset into training and testing datasets.

In [ ]:
 !python3.7 -m mimic3benchmark.scripts.split_train_and_test data/root/

### Step 5: Generate decompensation data
This step generates the decompensation data needed for StageNet

In [ ]:
!python3.7 -m mimic3benchmark.scripts.create_in_hospital_mortality data/root/ data/in-hospital-mortality/
!python3.7 -m mimic3benchmark.scripts.create_decompensation data/root/ data/decompensation/